In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
from imblearn.over_sampling import SMOTE

# 指定支持中文的字体，例如 'SimHei'
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决负号 '-' 显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

In [26]:
# 加载数据集
df = pd.read_csv('system_log.csv')

# 1. 将时间戳转换为datetime对象并排序
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

# 2. 识别数据采集的会话（Session）
time_diff = df['timestamp'].diff()
session_threshold = pd.Timedelta('1 minute')
df['session_id'] = (time_diff > session_threshold).cumsum()

# 3. 设置时间索引
# 这是最核心的改动。让所有操作都基于一个显式的时间索引。
df = df.set_index('timestamp')

# 4. 在每个会话内部进行滑动窗口处理
window_size = '10s' 
features_to_roll = [
    'mouse_left_click',
    'mouse_right_click',
    'mouse_scroll',
    'keyboard_counts',
    'mouse_distance',
    'bytes_sent_per_sec',
    'bytes_recv_per_sec',
    'packets_sent_per_sec',
    'packets_recv_per_sec',
    'read_bytes_per_sec',
    'write_bytes_per_sec'
]

for col in features_to_roll:
    new_col_name = f'{col}_freq'
    # 现在我们按session_id分组，并在时间索引上直接进行滚动计算
    # 注意，这里不再需要 on='timestamp' 参数，因为rolling操作默认就在索引上执行
    rolled_series = df.groupby('session_id')[col].rolling(window=window_size).sum()
    
    # 结果的索引是 (session_id, timestamp)，我们需要去掉第一层session_id来对齐
    df[new_col_name] = rolled_series.reset_index(level=0, drop=True)

df = df.reset_index()

# 6. 将时间戳转换为自第一个时间戳以来的总秒数
df['timestamp'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds()

# 7. 对 'label' 列进行编码
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
print("标签分布情况:\n", df['label'].value_counts())
num_classes = df['label'].nunique()

# 9. 处理滑动窗口产生的NaN值（通常是每个会话最开始的几个点）
df.fillna(0, inplace=True)

# 保存处理后的数据
df.to_csv('processed_system.csv', index=False)

标签分布情况:
 label
1    3870
3    3789
2    2245
0    1037
Name: count, dtype: int64


In [27]:
# 加载数据集
df2 = pd.read_csv('system_log_test.csv')

# 1. 将时间戳转换为datetime对象并排序
df2['timestamp'] = pd.to_datetime(df2['timestamp'])
df2 = df2.sort_values('timestamp').reset_index(drop=True)

# 2. 识别数据采集的会话（Session）
time_diff = df2['timestamp'].diff()
session_threshold = pd.Timedelta('1 minute')
df2['session_id'] = (time_diff > session_threshold).cumsum()

# 3. 设置时间索引
# 这是最核心的改动。让所有操作都基于一个显式的时间索引。
df2 = df2.set_index('timestamp')

# 4. 在每个会话内部进行滑动窗口处理
window_size = '10s' 
features_to_roll = [
    'mouse_left_click',
    'mouse_right_click',
    'mouse_scroll',
    'keyboard_counts',
    'mouse_distance',
    'bytes_sent_per_sec',
    'bytes_recv_per_sec',
    'packets_sent_per_sec',
    'packets_recv_per_sec',
    'read_bytes_per_sec',
    'write_bytes_per_sec'
]

for col in features_to_roll:
    new_col_name = f'{col}_freq'
    # 现在我们按session_id分组，并在时间索引上直接进行滚动计算
    # 注意，这里不再需要 on='timestamp' 参数，因为rolling操作默认就在索引上执行
    rolled_series = df2.groupby('session_id')[col].rolling(window=window_size).sum()
    
    # 结果的索引是 (session_id, timestamp)，我们需要去掉第一层session_id来对齐
    df2[new_col_name] = rolled_series.reset_index(level=0, drop=True)

df2 = df2.reset_index()

# 6. 将时间戳转换为自第一个时间戳以来的总秒数
df2['timestamp'] = (df2['timestamp'] - df2['timestamp'].min()).dt.total_seconds()

# 7. 对 'label' 列进行编码
label_encoder2 = LabelEncoder()
df2['label'] = label_encoder2.fit_transform(df2['label'])
print("标签分布情况:\n", df2['label'].value_counts())
num_classes = df2['label'].nunique()

# 9. 处理滑动窗口产生的NaN值（通常是每个会话最开始的几个点）
df2.fillna(0, inplace=True)

# 保存处理后的数据
df2.to_csv('processed_system_test.csv', index=False)

标签分布情况:
 label
1    548
3    506
2    455
0    399
Name: count, dtype: int64


In [28]:
# 分离特征 (X) 和目标 (y)
features_to_drop = [
    'label', 'timestamp', 'session_id',
    'mouse_distance', 'mouse_left_click', 'mouse_right_click', 'mouse_scroll', 'keyboard_counts',
    'bytes_sent_per_sec', 'bytes_recv_per_sec', 'packets_sent_per_sec',
    'packets_recv_per_sec', 'read_bytes_per_sec', 'write_bytes_per_sec'
]
X = df.drop(features_to_drop, axis=1)
y = df['label']
X_test = df2.drop(features_to_drop, axis=1)
y_test = df2['label']
print("用于训练的特征列信息:")
X.info()

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X, y)
print("\n--- SMOTE 应用后 ---")
print("重采样后训练集中的标签分布:")
print(pd.Series(y_train_res).value_counts())

用于训练的特征列信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10941 entries, 0 to 10940
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   cpu_percent                10941 non-null  float64
 1   ram_percent                10941 non-null  float64
 2   gpu_percent                10941 non-null  float64
 3   gpu_vram_percent           10941 non-null  float64
 4   mouse_left_click_freq      10941 non-null  float64
 5   mouse_right_click_freq     10941 non-null  float64
 6   mouse_scroll_freq          10941 non-null  float64
 7   keyboard_counts_freq       10941 non-null  float64
 8   mouse_distance_freq        10941 non-null  float64
 9   bytes_sent_per_sec_freq    10941 non-null  float64
 10  bytes_recv_per_sec_freq    10941 non-null  float64
 11  packets_sent_per_sec_freq  10941 non-null  float64
 12  packets_recv_per_sec_freq  10941 non-null  float64
 13  read_bytes_per_sec_freq    10941 n

In [29]:
model = XGBClassifier(
    objective='multi:softprob',  # 明确指定为多分类任务
    num_class=num_classes,       # 明确告知类别的数量
    eval_metric='mlogloss'       
)
# 评估方法一: 5折交叉验证
print("--- 1. 执行5折交叉验证 ---")
# 使用 cross_val_score 函数进行交叉验证，cv=5 表示5折
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"交叉验证的每次准确率分数: {cv_scores}")
print(f"交叉验证的平均准确率: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})\n")
# 在训练集上训练模型，用于后续的评估
model.fit(X_train_res, y_train_res, verbose=False)

# 在测试集上进行预测
y_pred = model.predict(X_test)

--- 1. 执行5折交叉验证 ---
交叉验证的每次准确率分数: [0.99451804 0.94515539 0.99040219 0.99040219 0.97120658]
交叉验证的平均准确率: 0.9783 (+/- 0.0185)



In [30]:
# 评估方法二: 分类报告
print("--- 2. 分类报告 ---")
# 获取原始的类别名称，用于报告显示
target_names = label_encoder.classes_
# 打印每个类别的精确率、召回率和F1分数
print(classification_report(y_test, y_pred, target_names=target_names))

--- 2. 分类报告 ---
              precision    recall  f1-score   support

      coding       1.00      0.99      1.00       399
      gaming       1.00      1.00      1.00       548
        idle       0.94      0.93      0.94       455
       video       0.94      0.95      0.94       506

    accuracy                           0.97      1908
   macro avg       0.97      0.97      0.97      1908
weighted avg       0.97      0.97      0.97      1908



In [31]:
# 评估方法三: 混淆矩阵可视化
print("--- 3. 生成混淆矩阵图 (confusion_matrix.png) ---")
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('混淆矩阵 (Confusion Matrix)', fontsize=16)
plt.ylabel('真实标签 (Actual Label)', fontsize=12)
plt.xlabel('预测标签 (Predicted Label)', fontsize=12)
# 保存图像到文件
plt.savefig('confusion_matrix.png')
plt.close() # 关闭图像，防止显示混乱
print("混淆矩阵图已保存。\n")

--- 3. 生成混淆矩阵图 (confusion_matrix.png) ---
混淆矩阵图已保存。



In [32]:
# 评估方法四: 特征重要性可视化
print("--- 4. 生成特征重要性图 (feature_importance.png) ---")
feature_importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
# 按重要性得分降序排列
importance_df = importance_df.sort_values(by='Importance', ascending=False)
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
plt.title('特征重要性 (Feature Importance)', fontsize=16)
plt.xlabel('重要性得分 (Importance Score)', fontsize=12)
plt.ylabel('特征 (Features)', fontsize=12)
plt.tight_layout() # 调整布局以防标签重叠
# 保存图像到文件
plt.savefig('feature_importance.png')
plt.close() # 关闭图像
print("特征重要性图已保存。\n")

print("所有评估已完成。")

--- 4. 生成特征重要性图 (feature_importance.png) ---


C:\Users\GUPID\AppData\Local\Temp\ipykernel_38900\3862479974.py:9: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')


特征重要性图已保存。

所有评估已完成。


In [33]:
# 保存模型到文件
joblib.dump(model, 'xgboost_model.joblib')

# 保存 LabelEncoder 到文件
joblib.dump(label_encoder, 'label_encoder.joblib')

print("模型已保存为 'xgboost_model.joblib'")
print("编码器已保存为 'label_encoder.joblib'")

模型已保存为 'xgboost_model.joblib'
编码器已保存为 'label_encoder.joblib'
